In [ ]:
import os
import pandas as pd
import numpy as np

class OutlierReplaceSystem:
    def __init__(self, df_final_path: str, column: str, mask: pd.Series):
        self.df_final_path = df_final_path
        self.column = column
        self.mask = mask

        if os.path.exists(self.df_final_path):
            self.df_final = pd.read_csv(self.df_final_path)

        else:
            return "Final dataset not found"

    def method(self, tool_name: str, config: dict):
        if tool_name == "Mean":
            self._mean()

        elif tool_name == "Median":
            self._median()

        elif tool_name == "constant":
            self._constant(config)

        elif tool_name == "ffill":
            self._ffill()

        elif tool_name == "bfill":
            self._bfill()

        else:
            return "Invalid replacement method"

        self.df_final.to_csv(self.df_final_path, index=False)
        return self.df_final

    def _target_index(self):
        nan_mask = self.df_final[self.column].isna()
        combined_mask = nan_mask | self.mask
        return combined_mask

    def _mean(self):
        combined_mask = self._target_index()
        mean_val = self.df_final[self.column].mean()
        self.df_final.loc[combined_mask, self.column] = mean_val

    def _median(self):
        combined_mask = self._target_index()
        median_val = self.df_final[self.column].median()
        self.df_final.loc[combined_mask, self.column] = median_val

    def _constant(self, config: dict):
        value = config.get("value")
        if value is None:
            return "value required"
        combined_mask = self._target_index()
        self.df_final.loc[combined_mask, self.column] = value\

    def _ffill(self):
        combined_mask = self._target_index()
        self.df_final.loc[combined_mask, self.column] = np.nan
        self.df_final[self.column] = self.df_final[self.column].fillna(method="ffill")

    def _bfill(self):
        combined_mask = self._target_index()
        self.df_final.loc[combined_mask, self.column] = np.nan
        self.df_final[self.column] = self.df_final[self.column].fillna(method="bfill")